In [1]:
import os
import athina
from athina.evals import DoesResponseAnswerQuery, ContextContainsEnoughInformation, Faithfulness
from athina.loaders import Loader
from athina.keys import AthinaApiKey, OpenAiApiKey
from athina.datasets import yc_query_mini
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

# Configure API keys
OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

#### Load a dataset
You can load a dataset using JSON, CSV, or a list of dictionaries

In [2]:
raw_data = yc_query_mini.data

dataset = Loader().load_dict(raw_data)
pd.DataFrame(dataset)

,query,context,response,expected_response
0,What are some successful companies that went t...,[Y Combinator has invested in companies in var...,"Airbnb, Dropbox, Stripe, Reddit, Coinbase, Ins...",Airbnb and Stripe are 2 of the successful comp...
1,In which city is YC located?,"[Y Combinator is located in Mountain View, Cal...",Y Combinator is located in San Francisco,"YC is located in Mountain View, California."
2,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,YC invests $150k for 7%.,I cannot answer this question as I do not have...
3,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,I cannot answer this question as I do not have...,I cannot answer this question as I do not have...
4,Who founded YC and when was it founded?,[Y Combinator was founded in March 2005 by Pau...,Y Combinator was founded in 2005,Y Combinator was founded in March 2005 by Paul...
5,Does Y Combinator invest in startups outside t...,[Y Combinator invests in startups from all ove...,"Yes, Y Combinator invests in international sta...","Yes, Y Combinator invests in startups from all..."
6,How much does YC invest in startups?,[YC invests $150k for 7%.],$150k,YC invests $150k for 7%.
7,What is YC's motto?,[Y Combinator's motto is 'Make something peopl...,Make something people want,Make something people want


#### Run the evaluations
- Configure an evaluation suite (choose from 50+ preset evals, or write a custom eval)
- Run the evals!

In [3]:
# Configure your evaluation suite
eval_model = "gpt-4o"
eval_suite = [
    DoesResponseAnswerQuery(model=eval_model),
    Faithfulness(model=eval_model),
    ContextContainsEnoughInformation(model=eval_model),
]

# Run the evaluation suite
athina.run(
    evals=eval_suite,
    data=dataset,
    max_parallel_evals=10
)

You can view your dataset at: https://app.athina.ai/develop/b38ff887-52d6-451a-9568-6ae35ac7134e


,query,context,response,expected_response,Does Response Answer Query passed,Faithfulness passed,Context Contains Enough Information passed
0,What are some successful companies that went t...,[Y Combinator has invested in companies in var...,"Airbnb, Dropbox, Stripe, Reddit, Coinbase, Ins...",Airbnb and Stripe are 2 of the successful comp...,1.0,0.0,0.0
1,In which city is YC located?,"[Y Combinator is located in Mountain View, Cal...",Y Combinator is located in San Francisco,"YC is located in Mountain View, California.",0.0,0.0,1.0
2,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,YC invests $150k for 7%.,I cannot answer this question as I do not have...,1.0,0.0,0.0
3,How much equity does YC take?,[Y Combinator invests $500k in 200 startups tw...,I cannot answer this question as I do not have...,I cannot answer this question as I do not have...,0.0,1.0,0.0
4,Who founded YC and when was it founded?,[Y Combinator was founded in March 2005 by Pau...,Y Combinator was founded in 2005,Y Combinator was founded in March 2005 by Paul...,0.0,1.0,1.0
5,Does Y Combinator invest in startups outside t...,[Y Combinator invests in startups from all ove...,"Yes, Y Combinator invests in international sta...","Yes, Y Combinator invests in startups from all...",1.0,1.0,1.0
6,How much does YC invest in startups?,[YC invests $150k for 7%.],$150k,YC invests $150k for 7%.,0.0,1.0,1.0
7,What is YC's motto?,[Y Combinator's motto is 'Make something peopl...,Make something people want,Make something people want,1.0,1.0,1.0
